In [24]:
from itertools import combinations
from collections import Counter
import os
import requests

"""
Скрипт виконує аналіз ринкового кошика для заданого набору замовлень, ідентифікує часті набори товарів та генерує асоціативні правила.

"""
def download_and_parse_document(file_name, document_url, max_orders=None):
    """
    Завантажує документ за вказаною URL-адресою та розбиває його на список замовлень.

    Параметри:
    - file_name (str): Назва файлу для завантаження або читання.
    - document_url (str): URL-адреса документа для завантаження.
    - max_orders (int, необов'язковий): Максимальна кількість замовлень для обробки. За замовчуванням дорівнює None (обробляються всі замовлення).

    Повертає:
    - список: Список замовлень.
    """

    if os.path.exists(file_name):
        print(f'The file {file_name} already exists. Skipping download.')
    else:
        response = requests.get(document_url)
        if response.status_code == 200:
            with open(file_name, 'wb') as f:
                f.write(response.content)

    content = open(file_name, 'r').read()
    orders = content.split('\n\n')
    list_prod = []

    for i, order in enumerate(orders):
        products = order.strip().split('@@@')
        list_prod.append(products)
        if max_orders and i + 1 >= max_orders:
            break

    return list_prod

def count_prod(prod):
    """
    Підраховує кількість кожного товару в списку замовлень та повертає словник з підрахунками товарів.

    Параметри:
    - prod (список): Список замовлень.

    Повертає:
    - словник: Словник товарів з їх сумарною кількістю.
    """
    product_counts = {}

    for order in prod:
        for product in order:
            product = product.strip()
            product_counts[product] = product_counts.get(product, 0) + 1
    return product_counts

def support(zakaz, min_support, min_confi):
    """
    Виконує аналіз кошика для списку замовлень та виводить асоціативні правила на основі заданих порогових значень.

    Параметри:
    - zakaz (список): Список замовлень для аналізу.
    - min_support (float): Мінімальний поріг підтримки для наборів товарів.
    - min_confi (float): Мінімальний поріг впевненості для асоціативних правил.

    Виводить пари продуктів, які відповідають заданим пороговим значенням:
    """
    def generate_unique_pairs(zakaz):
        """
        Створює кортеж з унікальними парами продуктів у списку замовлень.

        Параметри:
        - zakaz (список): Список замовлень для аналізу.

        Повертає:
        - кортеж унікальних пар товарів
        """
        unique_pairs = set()

        for order in zakaz:
            unique_pairs.update(combinations(sorted(order), 2))

        return  unique_pairs

    pair_counter = Counter()

    for pair in generate_unique_pairs(zakaz):
        for order in zakaz:
            if all(product in order for product in pair):
                pair_counter[pair] += 1

    prod_count=count_prod(zakaz)

    for pair, count in sorted(pair_counter.items(), key=lambda x: x[1], reverse=True):
        support = (pair_counter[pair] / len(zakaz)) * 100
        confidence1 = (pair_counter[pair] / prod_count.get(pair[0], 1)) * 100
        confidence2 = (pair_counter[pair] / prod_count.get(pair[1], 1)) * 100
        if support>min_support and confidence1>min_confi:
            print(f'{(pair[0])} => {(pair[1])} (confidence: {round(confidence1,2)}%, support: {round(support,2)}%)')
        if support>min_support and confidence2>min_confi:
            print(f'{(pair[1])} => {(pair[0])} (confidence: {round(confidence2,2)}%, support: {round(support,2)}%)')

#Приклад використання

file_name = 'orders.txt'
document_url = 'https://drive.google.com/uc?id=1IOPTVq2ooQfZRkF3rAjGkTjRtbotG7FF'
orders=download_and_parse_document(file_name, document_url, 1000)
support(orders, 2, 20)

The file orders.txt already exists. Skipping download.
Bag of Organic Bananas => Organic Strawberries (confidence: 24.51%, support: 2.5%)
Organic Strawberries => Bag of Organic Bananas (confidence: 24.51%, support: 2.5%)
Organic Strawberries => Banana (confidence: 22.55%, support: 2.3%)
Bag of Organic Bananas => Organic Hass Avocado (confidence: 21.57%, support: 2.2%)
Organic Hass Avocado => Bag of Organic Bananas (confidence: 37.29%, support: 2.2%)
